<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH19_04_BERT%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B6%9C_%ED%82%A4%EB%B2%84%ED%8A%B8(KeyBERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 우선 SBERT를 위한 패키지 설치 필요
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 1. 기본 KeyBERT
-----------------

In [2]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [3]:
doc = """
         Supervised learning is the machine learning task of 
         learning a function that maps an input to an output based 
         on example input-output pairs.[1] It infers a function 
         from labeled training data consisting of a set of 
         training examples.[2] In supervised learning, each 
         example is a pair consisting of an input object 
         (typically a vector) and a desired output value (also 
         called the supervisory signal). A supervised learning 
         algorithm analyzes the training data and produces an 
         inferred function, which can be used for mapping new 
         examples. An optimal scenario will allow for the algorithm 
         to correctly determine the class labels for unseen 
         instances. This requires the learning algorithm to  
         generalize from the training data to unseen situations 
         in a 'reasonable' way (see inductive bias).
      """

Scikit-learn의 CountVectorizer를 사용하여 단어 추출. n_gram_range 인자를 사용하여 3개의 단어를 한 묶음으로 간주하는 trigram 추출함 

In [4]:
n_gram_range = (3, 3)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names_out()

print('trigram 개수:', len(candidates))
print('trigram 단어묶음 다섯개만 출력:', candidates[:5])

trigram 개수: 72
trigram 단어묶음 다섯개만 출력: ['algorithm analyzes training' 'algorithm correctly determine'
 'algorithm generalize training' 'allow algorithm correctly'
 'analyzes training data']


다음으로 문서로부터 추출한 키워드들을 SBERT를 통해 수치화함(인코딩)


In [5]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])  # 전체 문서 인코딩
candidate_embeddings = model.encode(candidates)  # trigram 키워드 인코딩

In [6]:
print(doc_embedding.shape)  # 768개 숫자로 인코딩됨

(1, 768)


In [7]:
doc_embedding[0][0]

-1.1052822

In [8]:
candidate_embeddings.shape  #

(72, 768)

전체 문서와 가장 유사한 키워드를 추출함. 키워드 중에 문서와 가장 유사한 키워드는 문서를 대표하는 키워드로 가정하고, 상위 5개 키워드를 출력함.

In [9]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords

['algorithm analyzes training',
 'learning algorithm generalize',
 'learning machine learning',
 'learning algorithm analyzes',
 'algorithm generalize training']

5개의 키워드가 출력되었는데 의미가 비슷해 보임. 당연히 이 키워드들이 문서를 가장 잘 나타내기 때문임.

좀더 다양한 의미의 키워드들은 문서를 잘 나타낼 가능성이 적을 수도 있음. 따라서 키워드의 정확성과 다양성 사이에는 균형이 필요함.

여기서는 다양한 키워드들을 얻기 위해 두가지 알고리즘을 사용함.

* Max Sum Similarity
* Maximal Marginal Relevance


### 2. Max Sum Similarity
--------------------
키워드간 거리가 최대화되는 키워드 쌍을 정의하는 함수를 이용함. 키워드간 유사성을 최소화하면서 문서와의 유사성을 극대화한다는 것임.

In [10]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
  # 문서와 각 키워드들 간의 유사도
  distances = cosine_similarity(doc_embedding, candidate_embeddings)

  # 각 키워드들 간의 유사도
  distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)

  # 코사인 유사도에 기반하여 키워드 중 상위 top_n개의 단어를 선택
  words_idx = list(distances.argsort()[0][-nr_candidates:])  # 문서와 키워드간 유사도가 큰 n개 키워드 선택
  words_vals = [candidates[index] for index in words_idx]  # 문서와 키워드간 유사도 가 큰 n개 키워드의 유사도 값 추출
  distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)] # 선정된 키워드들간 유사도 추출

  # 선택한 키워드들 중 가장 덜 유사한 키워드들 간 조합 계산
  min_sim = np.inf
  candidate = None
  for combination in itertools.combinations(range(len(words_idx)), top_n):
    sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
    if sim < min_sim:
      candidate = combination
      min_sim = sim
  return [words_vals[idx] for idx in candidate]  # 선정했던 10개 키워드 중 상호간 유사도가 적은 키워드 추출

In [11]:
# 상위 10개 키워드를 선택하고 이 중 서로 가장 유사성이 낮은 5개를 선택함
# 낮은 nr_candidates를 설정하면 결과는 출력된 키워드 5개는 기존의 코사인 유사도만 사용한 것과 아주 유사한 결과를 도출
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['requires learning algorithm',
 'signal supervised learning',
 'learning function maps',
 'algorithm analyzes training',
 'learning machine learning']

In [12]:
# 상위 20개 키워드 중 유사하지 안은 키워드 5개 뽑기
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=20)

['set training examples',
 'generalize training data',
 'requires learning algorithm',
 'supervised learning algorithm',
 'learning machine learning']

### 3. Maximal Marginal Relevance
----------------
결과를 다양화하는 다른 방법으로 MMR(Maximal Marginal Relevance)이 있음. MMR은 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화함. 

* 참고자료 : EmbedRank(https://arxiv.org/pdf/1801.04470.pdf) ==> 키워드를 다양화하는데 사용할 수 있는 알고리즘

프로세스

* step 1) 문서와 가장 유사한 키워드/키 프레이즈를 선택
* step 2) 문서와 유사하고 이미 선택된 키워드/키 프레이즈와 유사하지 않은 새로운 후보 선택

step2 계속 반복함





In [15]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):
  # 문서와 각 키워드 간 유사도를 구한 리스트
  word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

  # 각 키워드 간 유사도
  word_similarity = cosine_similarity(candidate_embeddings, candidate_embeddings)

  # 문서와 가장 높은 유사도를 가진 키워드 인덱스 추출
  keywords_idx = [np.argmax(word_doc_similarity)]

  # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스
  candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
  # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래 반복
  for _ in range(top_n - 1):
    candidate_similarities = word_doc_similarity[candidates_idx, :]
    target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

    # MMR 계산
    mmr = (1 - diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
    mmr_idx = candidates_idx[np.argmax(mmr)]

    # keywords & candidates 업데이트
    keywords_idx.append(mmr_idx)
    candidates_idx.remove(mmr_idx)

  return [words[idx] for idx in keywords_idx]

In [16]:
# 만약 상대적으로 낮은  diversity 값을 설정할 수록, 결과는 기존의 코사인 유사도만 사용한 것과 유사한 결과를 보임
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['algorithm generalize training',
 'supervised learning algorithm',
 'learning machine learning',
 'learning algorithm analyzes',
 'learning algorithm generalize']

In [17]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['algorithm generalize training',
 'labels unseen instances',
 'new examples optimal',
 'determine class labels',
 'supervised learning algorithm']